Setup:

In [1]:
#!sudo apt install -qq -y fluidsynth  # that's for Linux.  on Mac: "brew install fluidsynth"
#!pip install -Uqq pyfluidsynth pretty_midi wandb

In [2]:
data_source = "maestro"

if data_source == "shakespeare":
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
else:
    !wget -N https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
    !unzip -n -qq maestro-v3.0.0-midi.zip

--2023-11-25 17:33:59--  https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.215.207, 142.250.9.207, 74.125.136.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.215.207|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘maestro-v3.0.0-midi.zip’ not modified on server. Omitting download.



In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from glob import glob
import pathlib
import pretty_midi
from tqdm import tqdm
import numpy as np
import random 

In [4]:
use_wandb = True
if use_wandb:
  import wandb
  wandb.login()

wandb: Currently logged in as: drscotthawley. Use `wandb login --relogin` to force relogin


In [5]:
data_dir = pathlib.Path('maestro-v3.0.0')
filenames = glob(str(data_dir/'**/*.mid*'))
print('Number of files:', len(filenames))

Number of files: 1276


In [6]:
vocab_size = 128

def midi_file_to_tensor(midi_file):
    pm = pretty_midi.PrettyMIDI(midi_file) # read in the whole file. this is incredibly slow

    # Sort the notes first by start time (then by pitch if two notes start at the same time)
    sorted_notes = sorted(pm.instruments[0].notes, key=lambda note: (note.start, note.pitch))
    notes = torch.empty( (len(sorted_notes), 3), dtype=torch.float32 ) # allocate storage

    prev_start = sorted_notes[0].start
    for i, note in enumerate(sorted_notes):
        notes[i] = note.pitch #/ vocab_size  # normalized pitch
        notes[i, 1] = note.start - prev_start  # step, i.e. time since last note started
        notes[i, 2] = note.end - note.start    # duration
        prev_start = note.start

    return notes

In [7]:
#inspect one file
notes = midi_file_to_tensor(filenames[0])
pitches = notes[:,0].type(torch.long)  # just the pitch info
notes.shape, pitches.shape

(torch.Size([3816, 3]), torch.Size([3816]))

In [8]:
notes_list = torch.load('maestro3_tensor_list.pt')  # load from previous computation
#notes_list = torch.load('rastro-120bpm_16th_tensor_list.pt')  # load from previous computation
len(notes_list)

1276

In [9]:
def tl_to_notes(tensor_list, shuffle=False, delimit=True):
  "list of tensors (of arbitrary length, for each song) converted to one big long tensor of notes all running togehter"
  if shuffle:random.shuffle(tensor_list)
  if delimit:
    delimiter = torch.zeros(3)  # use all zeros to show ends of songs
    tensor_list = [element for item in tensor_list for element in (item, delimiter)]
  return torch.vstack(tensor_list).type(torch.float32)  # return one big tensor of floats

seed = 1337
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)

all_notes = tl_to_notes(notes_list, shuffle=True) 
all_pitches = all_notes[:,0].type(torch.long)  # just the pitch info
all_notes.shape, all_pitches.shape

(torch.Size([7041440, 3]), torch.Size([7041440]))

In [10]:
'''
def time_quantize(notes_tensor,  # a single song
                  time_res=0.008, # resolution in seconds.  8ms is from Google "This Time With Feeling" paper
                  t_max=1.0, # again, from Google paper. This will give us from 0 to 1 second. Anything beyond that gets clipped
                  use_buckets=True,
                 ):
    nt2 = notes_tensor.contiguous().clone()
    if use_buckets:
        bucket_vals = torch.arange(0, t_max, time_res)
        boundaries = torch.arange(time_res/2, t_max - time_res/2, time_res)
        inds = torch.bucketize(nt2[:,1:].contiguous(), boundaries)
        #nt2[:,1:] = bucket_vals[inds]
        nt2[:,1:] = inds  # time is now in divisions with resolution time_res 
    else:
        nt2[:,1:] = torch.clamp(torch.floor(nt2[:,1:]/time_res)*time_res, 0.0, t_max)
    return nt2

quant_notes_list = [time_quantize(q) for q in notes_list]
quant_notes = time_quantize(all_notes)
'''

'\ndef time_quantize(notes_tensor,  # a single song\n                  time_res=0.008, # resolution in seconds.  8ms is from Google "This Time With Feeling" paper\n                  t_max=1.0, # again, from Google paper. This will give us from 0 to 1 second. Anything beyond that gets clipped\n                  use_buckets=True,\n                 ):\n    nt2 = notes_tensor.contiguous().clone()\n    if use_buckets:\n        bucket_vals = torch.arange(0, t_max, time_res)\n        boundaries = torch.arange(time_res/2, t_max - time_res/2, time_res)\n        inds = torch.bucketize(nt2[:,1:].contiguous(), boundaries)\n        #nt2[:,1:] = bucket_vals[inds]\n        nt2[:,1:] = inds  # time is now in divisions with resolution time_res \n    else:\n        nt2[:,1:] = torch.clamp(torch.floor(nt2[:,1:]/time_res)*time_res, 0.0, t_max)\n    return nt2\n\nquant_notes_list = [time_quantize(q) for q in notes_list]\nquant_notes = time_quantize(all_notes)\n'

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if data_source == "shakespeare":
    # hyperparameters
    batch_size = 16 # how many independent sequences will we process in parallel?
    block_size = 32 # what is the maximum context length for predictions?
    max_iters = 5000
    eval_interval = 100
    learning_rate = 1e-3
    eval_iters = 200
    n_embd = 64
    n_head = 4
    n_layer = 4
    dropout = 0.1
else:
    # hyperparameters
    batch_size = 64 # how many independent sequences will we process in parallel?
    block_size = 256 # what is the maximum context length for predictions?
    max_iters = 5000
    eval_interval = 100
    learning_rate = 1e-3
    eval_iters = 200
    n_embd = 128 # 64
    n_head = 8 # 4
    n_layer = 4 * 4
    dropout = 0.1


config = {
  'learning_rate': learning_rate,
  'batch_size': batch_size,
  'block_size': block_size,
  'n_embd': n_embd,
  'n_head': n_head,
  'n_layer': n_layer,
  'dropout': dropout
}

Dataset creation

In [12]:
torch.manual_seed(1337)

# here are all the unique characters that occur in this text
#chars = sorted(list(set(text)))
#vocab_size = len(chars)
# create a mapping from characters to integers
#stoi = { ch:i for i,ch in enumerate(chars) }
#itos = { i:ch for i,ch in enumerate(chars) }
#encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
#decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string



# Train and test splits
#data = torch.tensor(encode(text), dtype=torch.long)
data = all_pitches
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
print("train_data.shape, val_data.shape =",train_data.shape, val_data.shape)


def augment_data(x,y): # db = datablock - a seqeunce , likely of length 1+block_size 
    x,y = x.clone(), y.clone()  # avoid in-place alterations of data
    pitch_shift = torch.randint(low=-6, high=6, size=(1,1,1), dtype=torch.long).item()
    x, y = [torch.clamp(q + pitch_shift, 0, vocab_size-1) for q in [x,y]] 
    return x,y

# data loading
def get_batch(split, debug=False):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = augment_data(x,y)
    x, y = x.to(device), y.to(device)
    if debug: print(f"get_batch: x.shape = {x.shape}, y.shape = {y.shape}")
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


train_data.shape, val_data.shape = torch.Size([6337296]) torch.Size([704144])


In [23]:
%%timeit 
x,y = get_batch('val')

1.57 ms ± 4.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
x,y = get_batch('train',debug=True)
print(f"B, T = {batch_size}, {block_size}")

get_batch: x.shape = torch.Size([64, 256]), y.shape = torch.Size([64, 256])
B, T = 64, 256


x is a sequence

In [14]:
x[0]

tensor([68, 67, 58, 75, 59, 68, 74, 67, 58, 75, 74, 59, 68, 67, 75, 58, 44, 54,
        92, 60, 93, 92, 91, 89, 87, 61, 52, 85, 87, 85, 83, 42, 82, 52, 80, 58,
        79, 80, 78, 76, 75, 73, 59, 75, 51, 47, 73, 71, 70, 68, 40, 66, 47, 64,
        56, 63, 61, 59, 58, 49, 46, 55, 56, 52, 51, 49, 47, 46, 44, 27, 43, 40,
        39, 37, 34, 20, 32, 27, 44, 59, 51, 56, 52, 61, 55, 59, 51, 56, 49, 64,
        55, 47, 56, 63, 44, 56, 40, 28, 55, 39, 46, 58, 56, 27, 88, 55, 49, 51,
        46, 87, 85, 83, 82, 80, 79, 56, 46, 51, 49, 76, 75, 73, 71, 70, 68, 59,
        51, 67, 64, 63, 58, 62, 61, 51, 56, 59, 44, 43, 53, 61, 59, 42, 59, 64,
        56, 63, 54, 52, 68, 58, 66, 59, 51, 43, 53, 59, 31, 49, 40, 61, 59, 42,
        54, 30, 58, 92, 52, 54, 49, 90, 88, 87, 85, 83, 82, 59, 54, 52, 49, 80,
        78, 76, 75, 73, 71, 63, 52, 54, 70, 68, 66, 61, 65, 64, 63, 47, 59, 64,
        54, 66, 68, 70, 71, 76, 75, 77, 73, 59, 71, 70, 52, 68, 49, 67, 64, 63,
        56, 61, 52, 60, 61, 42, 63, 64, 

y is x shifted back by one and including new data.
in this sense only y[:,-1] is the "next token" being predicted.

In [15]:
y[0]

tensor([67, 58, 75, 59, 68, 74, 67, 58, 75, 74, 59, 68, 67, 75, 58, 44, 54, 92,
        60, 93, 92, 91, 89, 87, 61, 52, 85, 87, 85, 83, 42, 82, 52, 80, 58, 79,
        80, 78, 76, 75, 73, 59, 75, 51, 47, 73, 71, 70, 68, 40, 66, 47, 64, 56,
        63, 61, 59, 58, 49, 46, 55, 56, 52, 51, 49, 47, 46, 44, 27, 43, 40, 39,
        37, 34, 20, 32, 27, 44, 59, 51, 56, 52, 61, 55, 59, 51, 56, 49, 64, 55,
        47, 56, 63, 44, 56, 40, 28, 55, 39, 46, 58, 56, 27, 88, 55, 49, 51, 46,
        87, 85, 83, 82, 80, 79, 56, 46, 51, 49, 76, 75, 73, 71, 70, 68, 59, 51,
        67, 64, 63, 58, 62, 61, 51, 56, 59, 44, 43, 53, 61, 59, 42, 59, 64, 56,
        63, 54, 52, 68, 58, 66, 59, 51, 43, 53, 59, 31, 49, 40, 61, 59, 42, 54,
        30, 58, 92, 52, 54, 49, 90, 88, 87, 85, 83, 82, 59, 54, 52, 49, 80, 78,
        76, 75, 73, 71, 63, 52, 54, 70, 68, 66, 61, 65, 64, 63, 47, 59, 64, 54,
        66, 68, 70, 71, 76, 75, 77, 73, 59, 71, 70, 52, 68, 49, 67, 64, 63, 56,
        61, 52, 60, 61, 42, 63, 64, 65, 

Model definition

In [16]:

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            #print("logits.shape =",logits.shape,", targets.shape =",targets.shape)
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens, temperature=1.0):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits/temperature, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx



Instantiate and get ready to run

In [17]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

3.232128 M parameters


In [18]:
def save_checkpoint(step, model, optimizer,loss, name):
    name = name + f'_{step}.pt'
    print("Saving checkpoint to",name)
    torch.save({
            'step': step,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, name)

def load_checkpoint(name, model, optimizer):
    checkpoint = torch.load(name)
    model.load_state_dict(checkpoint['model_state_dict'])
    m = model.to(device)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    step = checkpoint['step']
    loss = checkpoint['loss']
    return step, m, optimizer, loss

In [20]:
if use_wandb: wandb.init(project='musicbox_pitches', config=config)

# Do training

In [ ]:
max_iters = 30000
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if use_wandb: wandb.log(losses)
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 2.2965, val loss 2.3290
step 100: train loss 2.2877, val loss 2.3201
step 200: train loss 2.2877, val loss 2.3229
step 300: train loss 2.2903, val loss 2.3109
step 400: train loss 2.2993, val loss 2.3195
step 500: train loss 2.2968, val loss 2.3241


In [22]:
save_checkpoint(max_iters, model, optimizer,loss, f"pitches_only_model")

Saving checkpoint to pitches_only_model_30000.pt


In [ ]:
if use_wandb: wandb.finish()

Generate

In [ ]:

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
#context = torch.tensor([57, 93, 43, 79, 87, 55, 91, 38, 74], dtype=torch.long, device=device).unsqueeze(0)
context = x[0,:5].detach().type(torch.long).to(device=device).unsqueeze(0)
print("context =",context)
#print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))
pitch_list = m.generate(context, max_new_tokens=200)[0].tolist()
print("pitch_list =     ",pitch_list)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional
from IPython.display import Audio, display


def notes_arr_to_df(notes_arr) -> pd.DataFrame:
    columns = ['pitch','step','duration']
    df = pd.DataFrame(notes_arr, columns=columns)
    df["start"] = ""
    df["end"] = ""

    prev_start = 0
    #for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    for i, row in df.iterrows():
        start = prev_start + float(row['step'])
        df.at[i, 'start'] = start
        df.at[i, 'end'] = start + float(row['duration'])
        prev_start = start
    return df

def df_to_midi(
        notes_df: pd.DataFrame,
        out_file: str = '',  # output file to save to, if any
        instrument_name: str = 'Acoustic Grand Piano', # whatever you want to call this instrument
        velocity: int = 100,  # note loudness
    ) -> pretty_midi.PrettyMIDI:
    "converts a dataframe to valid midi"

    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(
        program=pretty_midi.instrument_name_to_program(
            instrument_name))

    prev_start = 0
    for i, note in notes_df.iterrows(): # this is a serial operation, not sure how to parallelize
        start = float(prev_start + note['step'])
        end = float(start + note['duration'])
        note = pretty_midi.Note(
            velocity=velocity,
            pitch=int(note['pitch']),
            start=start,
            end=end,
        )
        instrument.notes.append(note)
        prev_start = start

    pm.instruments.append(instrument)
    if out_file: pm.write(out_file)
    return pm

def plot_piano_roll(notes_df: pd.DataFrame, count: Optional[int] = None):
    "produce a piano roll plot"
    if count:
        title = f'First {count} notes'
    else:
        title = f'Whole track'
        count = len(notes_df['pitch'])
    plt.figure(figsize=(20, 4))
    plot_pitch = np.stack([notes_df['pitch'], notes_df['pitch']], axis=0)
    plot_start_stop = np.stack([notes_df['start'], notes_df['end']], axis=0)
    plt.plot(
        plot_start_stop[:, :count], plot_pitch[:, :count], color="b", marker=".")
    plt.xlabel('Time [s]')
    plt.ylabel('Pitch')
    ax = plt.gca()
    ax.set_ylim([0, vocab_size])
    _ = plt.title(title)
    plt.show()


def midi_to_audio(pm: pretty_midi.PrettyMIDI, seconds=30, sr=16000):
  "midi to audio, playable in notebook"
  waveform = pm.fluidsynth(fs=float(sr))
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*sr]
  return display(Audio(waveform_short, rate=sr))

def pitches_to_midi(pitch_list, seconds=30):
    notes_tensor = torch.zeros((len(pitch_list), 3)) + 0.25
    for i, p in enumerate(pitch_list):
        notes_tensor[i,0] = p
    notes_df = notes_arr_to_df(notes_tensor.cpu().detach().numpy())
    midi = df_to_midi(notes_df)
    plot_piano_roll(notes_df)
    audio_display = midi_to_audio(midi, seconds=seconds)
    return audio_display

print("len(x[0]) =",len(x[0]))
pitches_to_midi(x[0])

In [ ]:
for variation in range(4):
  pitch_list = m.generate(context, max_new_tokens=200, temperature=2.0)[0].tolist()
  pitches_to_midi(pitch_list[:len(x[0])])

In [ ]:
pitch_list = m.generate(context, max_new_tokens=2000)[0].tolist()
print("len(pitch_list) =",len(pitch_list))
pitches_to_midi(pitch_list, seconds=300)